#CEFET/RJ

**PPCIC - Programa de Pós-Graduação em Ciência da Computação**

**Mestrado em Ciência da Computação**

---

**Sistemas de Recomendação**

**Disciplina**: Aprendizado de Máquina

**Professor**: Eduardo Bezerra

**Aluno**: Janio de Souza Lima

#1 Métodos simplificados de recomendação

##1.1 Recomendação pela popularidade

Nesse primeiro exemplo, usaremos um conjunto de dados hipotéticos de assuntos de interesse de usuários que serão identificado pela sua posição na lista e em que cada lista interna representa os assuntos de interesse desse usuário. Por exemplo o usuário na primeira posição (`users_interests[0]`) é interessado nos assuntos da lista `['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra']`. Com base na frequência de interesses será analisada a popularidade dos assuntos, realizadas duas formas simplificadas de recomendação e implementadas outras formas mais avançadas nas próximas seções [Grus, 2016].

In [ ]:
import math
from collections import Counter, defaultdict

In [ ]:
#Dados de exemplo - Assuntos de interesse de Cientistas de Dados
users_interests = [
    ['Hadoop', 'Big Data', 'HBase', 'Java', 'Spark', 'Storm', 'Cassandra'],
    ['NoSQL', 'MongoDB', 'Cassandra', 'HBase', 'Postgres'],
    ['Python', 'scikit-learn', 'scipy', 'numpy', 'statsmodels', 'pandas'],
    ['R', 'Python', 'statistics', 'regression', 'probability'],
    ['machine learning', 'regression', 'decision trees', 'libsvm'],
    ['Python', 'R', 'Java', 'C++', 'Haskell', 'programming languages'],
    ['statistics', 'probability', 'mathematics', 'theory'],
    ['machine learning', 'scikit-learn', 'Mahout', 'neural networks'],
    ['neural networks', 'deep learning', 'Big Data', 'artificial intelligence'],
    ['Hadoop', 'Java', 'MapReduce', 'Big Data'],
    ['statistics', 'R', 'statsmodels'],
    ['C++', 'deep learning', 'artificial intelligence', 'probability'],
    ['pandas', 'R', 'Python'],
    ['databases', 'HBase', 'Postgres', 'MySQL', 'MongoDB'],
    ['libsvm', 'regression', 'support vector machines']
]

In [ ]:
#Definição de assuntos mais populares
popular_interests = Counter(interest
                            for user_interests in users_interests
                            for interest in user_interests).most_common()

A primeira recomendação, que seria possível por exemplo para um usuário novo para o qual não se tem nenhum conhecimento sobre seus assuntos de interesse, é a recomendação direta dos assuntos mais populares da lista.

In [ ]:
#Recomendação simplificada sem considerar interesses do usuário
n_recomendacoes = 5
popular_interests[0:n_recomendacoes]

[('Python', 4), ('R', 4), ('Big Data', 3), ('HBase', 3), ('Java', 3)]

A segunda abordagem, ainda muito simples, é realizar a sugestão dos assuntos mais populares que o usuário ainda não tem interesse.

In [ ]:
def most_popular_new_interests(user_interests, max_results=5):
    """
    Função recomendadora simples
    Considera apenas a popularidade dos temas
    """
    suggestions = [(interest, frequency)
                   for interest, frequency in popular_interests
                   if interest not in user_interests]

    return suggestions[:max_results]

In [ ]:
#Recomendação simplificada que sugere novos temas populares
most_popular_new_interests(users_interests[1], 5)

[('Python', 4), ('R', 4), ('Big Data', 3), ('Java', 3), ('statistics', 3)]

##1.2 Filtragem colaborativa baseada no usuário

Na abordagem anterior o maior foco foi na popularidade dos assuntos de forma geral, sem levar em conta os interesses do usuário para o qual se fazia a recomendação. O máximo que foi feito foi a remoção dos temas que já estavam registrados em seus interesses. Nessa seção, será usada uma abordagem que busca identificar a similaridade entre os usuários, para que as recomendações levem em consideração os interesses atuais do usuário e a similaridade com interesses de outros usuário para recomendar novos assuntos.

![](https://upload.wikimedia.org/wikipedia/commons/5/52/Collaborative_filtering.gif)

Figura 1 - Ilutração didática da filtragem colaborativa ([Wikipedia](https://en.wikipedia.org/wiki/Recommender_system))

In [ ]:
import numpy as np

In [ ]:
#As funções a seguir foram transcritas de Grus [2016]
def cosine_similarity(v, w):
  """
  Cálculo de similaridade entre usuários
  Baseado na técnica similaridade de cossenos
  """
  return np.dot(v, w) / math.sqrt(np.dot(v, v) * np.dot(w, w))


def make_users_interest_vector(user_interests):
  """
  Função cria um vetor booleano de 1 para interesse no assunto
  e 0 para ausência de interesse no assunto
  """
  return [1 if interest in user_interests else 0
            for interest in unique_interests]


def most_similar_users_to(user_id):
    pairs = [(other_user_id, similarity)
             for other_user_id, similarity in
             enumerate(user_similarities[user_id])
             if user_id != other_user_id and similarity > 0]
    return sorted(pairs,
                  key=lambda pair: pair[1],
                  reverse=True)


def user_based_suggestions(user_id, include_current_interests=False):
    suggestions = defaultdict(float)
    for other_user_id, similarity in most_similar_users_to(user_id):
        for interest in users_interests[other_user_id]:
            suggestions[interest] += similarity

    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

In [ ]:
#Organização dos interesses
unique_interests = sorted(list({interest
                                for user_interests in users_interests
                                for interest in user_interests}))

In [ ]:
#Criação de uma matriz booleana dos interesses
user_interest_matrix = list(map(make_users_interest_vector, users_interests))
user_interest_matrix = np.array(user_interest_matrix)

In [ ]:
#Cálculo da similaridade entre os usuários
user_similarities = [[cosine_similarity(interest_vector_i, interest_vector_j)
                      for interest_vector_j in user_interest_matrix]
                     for interest_vector_i in user_interest_matrix]

In [ ]:
#Analisar a similaridade entre os usuários 0 e 8
user_similarities[0][8]

0.1889822365046136

In [ ]:
#Análise dos usuários mais similares ao usuário 0
user_id = 0
most_similar_users_to(user_id)

[(9, 0.5669467095138409),
 (1, 0.3380617018914066),
 (8, 0.1889822365046136),
 (13, 0.1690308509457033),
 (5, 0.1543033499620919)]

In [ ]:
#Apresentar as sugestões para o usuário 0
user_based_suggestions(user_id)

[('MapReduce', 0.5669467095138409),
 ('MongoDB', 0.50709255283711),
 ('Postgres', 0.50709255283711),
 ('NoSQL', 0.3380617018914066),
 ('neural networks', 0.1889822365046136),
 ('deep learning', 0.1889822365046136),
 ('artificial intelligence', 0.1889822365046136),
 ('databases', 0.1690308509457033),
 ('MySQL', 0.1690308509457033),
 ('Python', 0.1543033499620919),
 ('R', 0.1543033499620919),
 ('C++', 0.1543033499620919),
 ('Haskell', 0.1543033499620919),
 ('programming languages', 0.1543033499620919)]

##1.3 Filtragem colaborativa baseada em itens

Um conceito importante relacionado aos sistemas recomendadores é a mineração de padrões frequentes que referem-se a identificação de relações recorrentes entre itens de um conjunto de dados [Han, 2012]. Um exemplo didático interessante, baseado em regras de associação, é a análise de cestas de mercado em que se recomendam itens de acordo com a a análise de padrões frequentes de itens que são adquiridos em conjunto por outros clientes, como é ilustrado na Figura 2. Há diversos algoritmos que realizam este tipo de tarefa de mineração de dados, mas nessa seção iniciaremos com a continuidade do exemplo anterior e a implementação simplificada de funções para recomendação colaborativa baseada em itens como apresentada por Grus [2016]. 

![Análise de cesta de mercado](https://ars.els-cdn.com/content/image/3-s2.0-B978012381479100006X-f06-01-9780123814791.jpg?_)

Figura 2 - Análise de cesta de mercado [Han, 2012]

In [ ]:
interest_user_matrix = [[user_interest_vector[j]
                         for user_interest_vector in user_interest_matrix]
                        for j, _ in enumerate(unique_interests)]

O código acima faz a transposição da matriz de interesses

In [ ]:
interest_user_matrix[0]

[1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]

Em seguida, na célula a seguir, é calculada a similaridade entre os interesses.

In [ ]:
interest_similarities = [[cosine_similarity(user_vector_i, user_vector_j)
                           for user_vector_j in interest_user_matrix]
                          for user_vector_i in interest_user_matrix]

In [ ]:
#As funções a seguir foram transcritas de Grus [2016]
def most_similar_interests_to(interest_id):
    similarities = interest_similarities[interest_id]
    pairs = [(unique_interests[other_interest_id], similarity)
             for other_interest_id, similarity in enumerate(similarities)
             if interest_id != other_interest_id and similarity > 0]
    
    return sorted(pairs,
                  key=lambda pair: pair[1],
                  reverse=True)


def item_based_suggestions(user_id, include_current_interests=False):
    suggestions = defaultdict(float)
    user_interest_vector = user_interest_matrix[user_id]

    for interest_id, is_interested in enumerate(user_interest_vector):
        if is_interested == 1:
            similar_interests = most_similar_interests_to(interest_id)
            for interest, similarity in similar_interests:
                suggestions[interest] += similarity

    suggestions = sorted(suggestions.items(),
                         key=lambda pair: pair[1],
                         reverse=True)

    if include_current_interests:
        return suggestions
    else:
        return [(suggestion, weight)
                for suggestion, weight in suggestions
                if suggestion not in users_interests[user_id]]

Aplicamos a seguir a função `most_similar_interests_to` para analisar a similaridade com um determinado assunto (item do conjunto de dados).

In [ ]:
unique_interests[0]

'Big Data'

In [ ]:
#Analisar similaridade com o assunto 'Big Data'
insterst_id=0 #Big Data
most_similar_interests_to(insterst_id)

[('Hadoop', 0.8164965809277261),
 ('Java', 0.6666666666666666),
 ('MapReduce', 0.5773502691896258),
 ('Spark', 0.5773502691896258),
 ('Storm', 0.5773502691896258),
 ('Cassandra', 0.4082482904638631),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('HBase', 0.3333333333333333)]

Por fim, realizamos a recomendação para um usuário baseado na similaridade de interesses.

In [ ]:
user_id = 0
item_based_suggestions(user_id)

[('MapReduce', 1.861807319565799),
 ('MongoDB', 1.3164965809277263),
 ('Postgres', 1.3164965809277263),
 ('NoSQL', 1.2844570503761732),
 ('MySQL', 0.5773502691896258),
 ('databases', 0.5773502691896258),
 ('Haskell', 0.5773502691896258),
 ('programming languages', 0.5773502691896258),
 ('artificial intelligence', 0.4082482904638631),
 ('deep learning', 0.4082482904638631),
 ('neural networks', 0.4082482904638631),
 ('C++', 0.4082482904638631),
 ('Python', 0.2886751345948129),
 ('R', 0.2886751345948129)]

#2 Algoritmos para implementação de sistemas recomendadores

As implementações das seções anteriores, apesar de funciorem com conjuntos de dados limitados, tem objetivo mais didático. Elas não tem capacidade de escalonamento, sobretudo por usarem estruturas de dados simples do Python com baixa perfomance em conjuntos de dados maiores.

##2.1 Mineração de padrões frequentes



**Frequent itemset e regras de associação**

* Itemset: uma combinação de itens existentes na base de dados.
* $support(A=>B) = P(A \cup B)$
* $confidence(A=>B) = P(B|A) = \frac{support(A \cup B)}{support(A)}$

Seção em desenvolvimento...

#Referências e links

**Referências**

Grus, Joel (2016). **Data Science do Zero:** Primeiras regras com Python. Traduzido por Welington Nascimento. Alta Books.

Han, Jiawei. Kamber, Micheline. Pei, Jian (2012) **Data Mining:** Concepts and techniques. Morgan Kaufmann, 3rd Edition.

McKinney, Wes (2018). **Python para Análise de Dados**: tratamento de dados com pandas, numpy e iPython. Traduzido por Lúcia A. Kinoshita. Novatec.

---
Conteúdos adicionais e explicações detalhadas do algoritmo em:

https://www.youtube.com/watch?v=4-dMKU9zThA&list=PLTy3TWJeueGw1eeP1cvaIiTzJrIljcVAq&index=14